In [97]:
import pandas as pd
from pathlib import Path

# Dataset analysis

## Counts by authority

In [103]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df["modern"] = (
    df["end-date"]
    .isna()
    .map({True: "Current authorities", False: "Former authorities"})
)

(
    df.pivot_table(
        "local-authority-code",
        index="local-authority-type-name",
        columns="modern",
        aggfunc="count",
    )
    .reset_index()
    .rename(
        columns={
            "local-authority-code": "Count",
            "local-authority-type-name": "Authority type",
        }
    )
    .fillna(0)
    .style.format({"Current authorities": int, "Former authorities": int})
    .hide_index()
)

Authority type,Current authorities,Former authorities
City corporation,1,0
Combined authority,10,0
County,24,3
London borough,32,0
Metropolitan district,36,0
NI district,11,25
Non-metropolitan district,181,23
Scottish unitary authority,32,0
Strategic Regional Authority,1,0
Unitary authority,59,2


## Unitary/lower tier and total counts

In [100]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df = df[df["current-authority"]]

df["lower_or_unitary"] = df["lower_or_unitary"].map({True: "Yes", False: "No"})

df = (
    df.pivot_table("local-authority-code", index="lower_or_unitary", aggfunc="count")
    .reset_index()
    .rename(
        columns={
            "local-authority-code": "Count",
            "lower_or_unitary": "Lower or unitary?",
        }
    )
    .fillna(0)
)

df
df = df.append(
    {"Lower or unitary?": "All", "Count": df["Count"].sum()}, ignore_index=True
)
df

,Lower or unitary?,Count
0,No,35
1,Yes,374
2,All,409


## Incomplete lookups

This are optional columns, and not entirely populated.

In [101]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df = df[df["current-authority"]]
items = []

cols = list(df.columns)
cols = cols[cols.index("BS-6879") :]


for c in cols:
    count = (~df[c].isna()).sum()
    items.append({"column": c, "complete": count})

pt = pd.DataFrame(items).sort_values("complete", ascending=False)
pt["%"] = pt["complete"] / len(df)
pt = pt[pt["%"] < 1]
pt.style.format({"%": "{:.1%}".format}).hide_index()

column,complete,%
BS-6879,407,99.5%
x,406,99.3%
y,406,99.3%
long,406,99.3%
lat,406,99.3%
old-register-and-code,404,98.8%
wdtk-id,404,98.8%
open-council-data-id,393,96.1%
gov-uk-slug,390,95.4%
os,380,92.9%
